<a href="https://colab.research.google.com/github/IvyCheng00/academic-projects/blob/main/rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **打造RAG系統**





In [ ]:
# AI生成

URL = "https://xxx"

!wget -O faiss_db.zip "$URL"

--2025-04-09 16:22:36--  https://drive.google.com/uc?export=download&id=154SJlk2AegRzXLsrmESBIShGp41D1ZF4
Resolving drive.google.com (drive.google.com)... 142.250.141.113, 142.250.141.102, 142.250.141.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=154SJlk2AegRzXLsrmESBIShGp41D1ZF4&export=download [following]
--2025-04-09 16:22:36--  https://drive.usercontent.google.com/download?id=154SJlk2AegRzXLsrmESBIShGp41D1ZF4&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.137.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9168 (9.0K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]   8.95K  --.-KB/s 

In [ ]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


In [ ]:
!pip install -U langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr

# 自訂E5 embedding類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
  def embed_documents(self, texts):
    texts = [f"passage: {t}" for t in texts]
    return super().embed_documents(texts)

  def embed_query(self, text):
    return super().embed_query(f"query: {text}")

# 載入faiss_db

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

# 設定LLM

In [ ]:
import os
from google.colab import userdata

In [ ]:
from posix import environ
api_key = userdata.get("OpenAI")

os.environ["OPENAI_API_KEY"] = api_key

base_url = "https://api.openai.com/v1"
model = "gpt-4o"

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url = base_url,
)

# prompt設計

In [ ]:
system_prompt = "請根據資料回答問題"

prompt_template = """
根據下列資料回答問題:
{retrieved_chunks}

使用者的問題是: {question}

請根據資料內容回覆，若資料不足就用機智（或搞笑）方式請使用者一起來討論、猜猜看～嘿嘿！
"""

# RAG回應

In [ ]:
chat_history = []

def chat_with_rag(user_input):
  global chat_history

  # 取得資料
  docs = retriever.get_relevant_documents(user_input)
  retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

  final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

  response = client.chat.completions.create(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": final_prompt},
      ]
  )

  answer = response.choices[0].message.content

  chat_history.append((user_input, answer))
  return answer

In [ ]:
print(chat_with_rag("請問xxx？"))

嘿嘿，你問對人啦！WOOZI可是我們SEVENTEEN的超級才子呢！根據資料，WOOZI的韓文名字是이지훈，中文名字是李知勳（注意，不要跟其他明星搞混哦！）他出生於1996年11月22日，是個火辣辣的射手座～而最厲害的是，他是我們Vocal Team的隊長，讓我們的歌聲如此美妙的幕後推手！出生地在韓國釜山廣域市，所以啊，他的靈魂中一定充滿著釜山的熱情和活力呢！想知道更多WOOZI的趣事，那就來跟我一起當個可愛又忠實的Carat吧！（抱可拉一起旋轉～）


# gradio

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("# RAG-robot")
  chatbot = gr.Chatbot()
  msg = gr.Textbox(placeholder="請輸入你的問題...")

  def respond(message, chat_history_local):
    response = chat_with_rag(message)
    chat_history_local.append((message, response))
    return "", chat_history_local

  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-35-b9268c78ebf7>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://321bc58fa697c5fcad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://321bc58fa697c5fcad.gradio.live
